In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cd /content/gdrive/MyDrive/trading_bot/Datos/crudos

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def process(folder, f_name, name, guardar=False):

  # cargo df
  f = f_name
  print(name)
  df = pd.read_csv(folder + '/' + f,header=1)
  df = df.drop('Unnamed: 0', 1)

  # filas con data
  above_resultado_NaN = df["Resultado"].notnull()
  df = df.loc[above_resultado_NaN,:]

  # selecciono columnas primitivas
  df = df.loc[:, ['Date', 'Open', 'High','Low','Close','Volume']]

  # formateo
  df['Open'] = df['Open'].str.replace(',', '.', regex=False).astype(float)
  df['High'] = df['High'].str.replace(',', '.', regex=False).astype(float)
  df['Low'] = df['Low'].str.replace(',', '.', regex=False).astype(float)
  df['Close'] = df['Close'].str.replace(',', '.', regex=False).astype(float)
  df['Volume'] = df['Volume'].str.replace(',', '.', regex=False).astype(int)
  df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%Y")

  # genero columnas derivadas
  df['Open_1'] = df.apply(lambda x: x['Open'] * 1.01, axis=1)
  df['mov_high'] = df.apply(lambda x: (x['High'] - x['Open_1']) / x['Open_1'] * 100, axis=1)
  df['mov_low'] = df.apply(lambda x: (x['Low'] - x['Open_1']) / x['Open_1'] * 100, axis=1)
  df['cierre'] = df.apply(lambda x: (x['Close'] - x['Open_1']) / x['Open_1'] * 100, axis=1)

  df['helper_column'] = df['Close'].shift(-1)
  df['gap'] = df.apply(lambda x: (x['Open'] - x['helper_column']) / x['helper_column'] * 100, axis=1)
  df = df.drop('helper_column', 1)

  # selecciono columnas finales
  df = df.loc[:, ['Date','gap','Open', 'Open_1', 'mov_high', 'mov_low', 'cierre', 'Volume','Close']]

  if guardar:
    df.to_csv(f'/content/gdrive/MyDrive/trading_bot/Datos/limpios/{name}')
  print("Exito") 

In [ ]:
folder = '/content/gdrive/MyDrive/trading_bot/Datos/crudos'

count = 0
for filename in os.listdir(folder):
  name = filename.split('-')[1].strip()
  process(folder ,filename, name, guardar=True)
  count= count + 1
  print("procesadas: ", count)

In [ ]:
def Q1(col):
  q1 = col.quantile(q=0.25)
  return q1

def Q2(col):
  q2 = col.quantile(q=0.5)
  return q2

def Q3(col):
  q3 = col.quantile(q=0.75)
  return q3

def L_sup(q1, q3):
  whis = 1.5
  l_sup = q3 + whis*(q3-q1)
  return l_sup

In [ ]:
summary = {'accion':[], 
           'estrategia': [],
           'TP': [],
           'SL': [],
           'TR':[],
           'sample' : [],
           'sample_per' : [] 
}

In [ ]:
def estrategias(folder, filename, instrumento):

  # cargo df
  f = filename
  #print(f)
  df = pd.read_csv(folder + '/' + f,header=0, dtype = {'gap': 'float64',
                                                       'Open' : 'float64', 
                                                       'Open_1':'float64',
                                                       'mov_high':'float64',
                                                       'mov_low':'float64',
                                                       'cierre':'float64',
                                                       'Volume':'int64',
                                                       'Close':'float64'})
  df = df.drop('Unnamed: 0', 1)
  df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")


  accion = [instrumento] * 3 
  summary['accion'].extend(accion)


  # Estrategia 0

  summary['estrategia'].append('estrategia_0')

  df_original = df.copy()
  n_0 = len(df_original)

  # mov_high > 1% 
  df_original = df_original.loc[df_original['mov_high'] >= 1, :]
  # gap < 1%
  df_original = df_original.loc[df['gap'] <= 1, :]

  # Precio > 25 
  df_original = df_original.loc[df_original['Close'] >= 25, :]
  #print(len(df_original) / n * 100)

  # Volumen > 500000
  df_original = df_original.loc[df_original['Volume'] >= 5000000, :]

  n_original = len(df_original)
  summary['sample'].append(n_original)

  p_sample_original = np.round((n_original / n_0) * 100, 2)
  summary['sample_per'].append(p_sample_original)
  

  # indicadores
  TP_original = np.round(df_original['mov_high'].mean(),2)
  summary['TP'].append(TP_original)
  SL_original = np.round(df_original['mov_low'].mean(),2)
  summary['SL'].append(SL_original)
  TR_original = np.round(df_original['mov_high'].std(),2)
  summary['TR'].append(TR_original)

  # Estrategia 1

  summary['estrategia'].append('estrategia_1')

  df_filtrada = df.copy()
  n_1 = len(df_filtrada)

  # datos extremos > mov_high >= 50%  
  df_filtrada = df_filtrada.loc[(df_filtrada['mov_high'] < L_sup(Q1(df_filtrada['mov_high']), Q3(df_filtrada['mov_high']))) & (df_filtrada['mov_high'] >= Q2(df_filtrada['mov_high'])), :]

  # gap < 75%
  df_filtrada = df_filtrada.loc[df_filtrada['gap'] <= Q3(df_filtrada['gap']), :]

  # Precio = Close >= 25%
  df_filtrada = df_filtrada.loc[df_filtrada['Close'] >= Q1(df_filtrada['Close']), :]

  # Volumen => 25%
  df_filtrada = df_filtrada.loc[df_filtrada['Volume'] >= Q1(df_filtrada['Volume']), :]

  n_filtrado = len(df_filtrada)
  summary['sample'].append(n_filtrado)
  p_sample_filtrado = np.round((n_filtrado / n_1) * 100, 2)
  summary['sample_per'].append(p_sample_filtrado)

  TP_filtros = np.round(df_filtrada['mov_high'].mean(),2)
  summary['TP'].append(TP_filtros)
  SL_filtros = np.round(df_filtrada['mov_low'].mean(),2)
  summary['SL'].append(SL_filtros)
  TR_filtros = np.round(df_filtrada['mov_high'].std(),2)
  summary['TR'].append(TR_filtros)

  # Estrategia 2

  summary['estrategia'].append('estrategia_2')

  g = df.groupby(pd.Grouper(key='Date', freq='6M'))
  dfs = [ group for _,group in g]
  df_6meses = dfs[-1]

  df_filtrada_6meses = df_6meses.copy()
  n_3 = len(df_filtrada_6meses)

  # datos extremos > mov_high >= 50%
  df_filtrada_6meses = df_filtrada_6meses.loc[(df_filtrada_6meses['mov_high'] < L_sup(Q1(df_filtrada_6meses['mov_high']), Q3(df_filtrada_6meses['mov_high']))) & (df_filtrada_6meses['mov_high'] >= Q2(df_filtrada_6meses['mov_high'])), :]

  # gap < 75%
  df_filtrada_6meses= df_filtrada_6meses.loc[df_filtrada_6meses['gap'] <= Q3(df_filtrada_6meses['gap']), :]
  #print(len(df_filtrada_6meses) / n * 100)

  # Precio = Close >= 25%
  df_filtrada_6meses = df_filtrada_6meses.loc[df_filtrada_6meses['Close'] >= Q1(df_filtrada_6meses['Close']), :]

  # Volumen => 25%
  df_filtrada_6meses = df_filtrada_6meses.loc[df_filtrada_6meses['Volume'] >= Q1(df_filtrada_6meses['Volume']), :]

  n_filtrada_6meses = len(df_filtrada_6meses)
  summary['sample'].append(n_filtrada_6meses)
  p_sample_filtrada_6meses = np.round((n_filtrada_6meses/ n_3) * 100,2)
  summary['sample_per'].append(p_sample_filtrada_6meses)

  # indicadores
  TP_6meses = np.round(df_filtrada_6meses['mov_high'].mean(),2)
  summary['TP'].append(TP_6meses)
  SL_6meses = np.round(df_filtrada_6meses['mov_low'].mean(),2)
  summary['SL'].append(SL_6meses)
  TR_6meses = np.round(df_filtrada_6meses['mov_high'].std(),2)
  summary['TR'].append(TR_6meses)


In [ ]:
folder = '/content/gdrive/MyDrive/trading_bot/Datos/limpios'

for filename in os.listdir(folder):
  instrumento = filename.split('.')[0]
  estrategias(folder ,filename, instrumento)


In [ ]:
df_estrategias = pd.DataFrame.from_dict(summary)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_estrategias.groupby(['accion','estrategia']).first()

TP    SL     TR  sample  sample_per
accion estrategia                                         
AMAT   estrategia_0  2.33 -1.79   1.49     198       15.74
       estrategia_1  1.02 -1.93   0.79     239       19.00
       estrategia_2  0.98 -1.91   0.63      21       18.10
AMC    estrategia_0  9.33 -6.70  11.68      25        1.99
       estrategia_1  2.39 -2.58   1.48     226       17.97
       estrategia_2  8.83 -7.17   4.45      21       18.10
AMD    estrategia_0  2.60 -1.96   1.52      59       23.32
       estrategia_1  1.55 -2.37   0.99      48       18.97
       estrategia_2  1.42 -1.87   0.81      20       17.39
BA     estrategia_0  2.80 -2.23   2.01     127       10.10
       estrategia_1  0.69 -1.94   0.58     228       18.12
       estrategia_2  0.87 -1.85   0.62      22       18.97
BABA   estrategia_0  1.97 -1.53   0.90     149       11.84
       estrategia_1  0.73 -1.84   0.67     244       19.40
       estrategia_2  0.21 -1.89   0.54      19       16.38
BAC    estrategia_0  1.78 -1.52   0.73      84        6.68
       estrategia_1  0.57 -1.70   0.61     242       19.24
       estrategia_2  0.80 -1.76   0.52      23       19.83
C      estrategia_0  1.63 -1.60   0.55      40       15.81
       estrategia_1  1.04 -1.70   0.66      52       20.55
       estrategia_2  0.88 -1.79   0.50      24       20.87
CVX    estrategia_0  2.18 -1.75   1.18      90        7.15
       estrategia_1  0.20 -1.55   0.41     230       18.28
       estrategia_2  0.59 -1.61   0.55      24       20.69
DAL    estrategia_0  2.36 -1.60   1.20      53       20.95
       estrategia_1  1.47 -2.18   0.88      50       19.76
       estrategia_2  1.02 -2.22   0.82      22       19.13
DKNG   estrategia_0  3.65 -2.91   2.50     121       28.47
       estrategia_1  3.64 -2.94   2.04      82       19.29
       estrategia_2  2.55 -2.98   1.19       5       16.67
GE     estrategia_0  2.41 -2.05   1.64     246       19.55
       estrategia_1  1.04 -2.00   0.84     242       19.24
       estrategia_2  1.04 -1.74   0.62      22       18.97
GILD   estrategia_0  1.83 -1.28   0.95      29       11.42
       estrategia_1  0.73 -1.60   0.66      51       20.08
       estrategia_2  0.77 -1.42   0.68      24       20.87
GM     estrategia_0  2.21 -1.59   1.13     135       10.73
       estrategia_1  0.49 -1.76   0.54     227       18.04
       estrategia_2  0.85 -2.02   0.73      21       18.10
HOOD   estrategia_0  5.56 -4.38   6.45      14       41.18
       estrategia_1  5.45 -5.81   2.24       5       14.71
       estrategia_2  5.45 -5.81   2.24       5       15.62
INTC   estrategia_0  2.03 -1.66   1.35     150       11.92
       estrategia_1  0.65 -1.91   0.59     238       18.92
       estrategia_2  0.39 -1.78   0.54      22       18.97
LI     estrategia_0  4.32 -3.46   3.38      53       20.38
       estrategia_1  4.22 -3.38   1.97      51       19.62
       estrategia_2  4.01 -3.87   2.03      23       19.83
MRNA   estrategia_0  4.20 -2.74   3.00     101       39.92
       estrategia_1  5.06 -2.27   2.38      52       20.55
       estrategia_2  5.13 -1.80   2.19      22       19.30
MS     estrategia_0  2.14 -1.77   1.45     156       12.40
       estrategia_1  0.69 -1.67   0.64     239       19.00
       estrategia_2  0.74 -1.57   0.61      23       19.83
MU     estrategia_0  2.31 -1.84   1.34     242       19.24
       estrategia_1  1.55 -2.09   0.93     244       19.40
       estrategia_2  0.98 -2.29   0.71      22       18.97
NCLH   estrategia_0  2.89 -2.50   2.18      44        3.50
       estrategia_1  0.99 -1.70   0.78     221       17.57
       estrategia_2  1.90 -2.37   0.85      22       18.97
NIO    estrategia_0  4.14 -3.10   2.79      84       11.43
       estrategia_1  4.22 -3.22   2.09     139       18.91
       estrategia_2  3.96 -2.01   1.53      19       20.43
NVDA   estrategia_0  2.46 -2.00   1.68     238       18.92
       estrategia_1  1.37 -2.12   0.94     244       19.40
       estrategia_2  1.37 -1.93   0.93      22       18.97
OXY    estr

In [ ]:
df_test_bt = df_estrategias.loc[df_estrategias['accion'].isin(['PENN','TSM','CSCO','XPEV','COIN','RIOT','LI', 'MU', 'AFRM', 'UAL' ]),:]

In [ ]:
df_test_bt.groupby(['accion','estrategia']).first()

TP    SL    TR  sample  sample_per
accion estrategia                                        
LI     estrategia_0  4.32 -3.46  3.38      53       20.38
       estrategia_1  4.22 -3.38  1.97      51       19.62
       estrategia_2  4.01 -3.87  2.03      23       19.83
MU     estrategia_0  2.31 -1.84  1.34     242       19.24
       estrategia_1  1.55 -2.09  0.93     244       19.40
       estrategia_2  0.98 -2.29  0.71      22       18.97
RIOT   estrategia_0  6.84 -5.21  5.67      50        3.97
       estrategia_1  5.97 -4.44  2.89     235       18.68
       estrategia_2  7.06 -4.25  3.03      22       18.97
UAL    estrategia_0  3.26 -2.47  2.46     138       10.97
       estrategia_1  0.97 -1.71  0.71     226       17.97
       estrategia_2  1.62 -2.47  1.02      22       18.97
XPEV   estrategia_0  4.08 -3.57  3.28      72       30.00
       estrategia_1  4.41 -3.67  1.94      46       19.17
       estrategia_2  4.83 -2.25  2.03      24       20.69